In [1]:
from dask.distributed import Client, progress
# Connect to the Dask cluster
client = Client('localhost:8786')

# Display the client details
print(client)

# Optionally, print the dashboard link
print(f"Dask dashboard available at: {client.dashboard_link}")


<Client: 'tcp://10.249.2.147:8786' processes=5 threads=1280, memory=2.33 TiB>
Dask dashboard available at: http://localhost:8787/status


2024-11-19 17:45:57,031 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [2]:

from prefect import flow, task
from prefect_dask import DaskTaskRunner


#only works when running on slac ip
# prefect config set PREFECT_API_URL="https://ard-modeling-service.slac.stanford.edu/api" 

# Define tasks
@task
def fetch_data(dataset: str) -> str:
    """
    Simulate fetching data for a given dataset.
    """
    print(f"Fetching data for {dataset}")
    return f"data_{dataset}"

@task
def process_data(data: str) -> str:
    """
    Simulate processing of data.
    """
    print(f"Processing {data}")
    return f"processed_{data}"

@task
def save_results(result: str) -> None:
    """
    Simulate saving processed data.
    """
    print(f"Saving {result}")

# Define flow
@flow(task_runner=DaskTaskRunner(address="tcp://localhost:8786"))
def parallel_workflow(datasets: list):
    """
    A workflow that fetches, processes, and saves datasets in parallel.
    """
    # Fetch data in parallel
    raw_data = [fetch_data.submit(dataset) for dataset in datasets]
    
    # Process data in parallel
    processed_data = [process_data.submit(data) for data in raw_data]
    
    # Save results in parallel
    [save_results.submit(result) for result in processed_data]

# List of datasets
datasets = ["dataset1", "dataset2", "dataset3", "dataset4"]

# Run the workflow
parallel_workflow(datasets)

17:44:33.139 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8323
See https://docs.prefect.io/3.0/manage/self-host#self-host-a-prefect-server for more information on running a dedicated Prefect server.

17:44:36.406 | INFO    | prefect.engine - Created flow run 'accelerated-nyala' for flow 'parallel-workflow'

17:44:36.444 | INFO    | prefect.task_runner.dask - Connecting to an existing Dask cluster at tcp://localhost:8786

17:44:38.304 | WARNING | Flow run 'accelerated-nyala' - A future was garbage collected before it resolved. Please call `.wait()` or `.result()` on futures to ensure they resolve.

17:44:38.306 | WARNING | Flow run 'accelerated-nyala' - A future was garbage collected before it resolved. Please call `.wait()` or `.result()` on futures to ensure they resolve.

...

17:44:38.312 | WARNING | Flow run 'accelerated-nyala' - A future was garbage collected before it resolved. Please call `.wait()` or `.result()` on futures to ensure they resolve.

17:44:38.313 | WARNING | Flow run 'accelerated-nyala' - A future was garbage collected before it resolved. Please call `.wait()` or `.result()` on futures to ensure they resolve.

17:44:38.314 | WARNING | Flow run 'accelerated-nyala' - A future was garbage collected before it resolved. Please call `.wait()` or `.result()` on futures to ensure they resolve.

17:44:38.315 | WARNING | Flow run 'accelerated-nyala' - A future was garbage collected before it resolved. Please call `.wait()` or `.result()` on futures to ensure they resolve.

17:44:38.444 | INFO    | Flow run 'accelerated-nyala' - Finished in state Completed()

In [3]:
import dask
from prefect import flow, task
from prefect_dask import DaskTaskRunner, get_dask_client

@task
def compute_task():
    with get_dask_client() as client:
        df = dask.datasets.timeseries("2000", "2001", partition_freq="4w")
        summary_df = df.describe().compute().result()
    return summary_df

@flow(task_runner=DaskTaskRunner(address="tcp://localhost:8786"))
def dask_flow():
    prefect_future = compute_task.submit()
    return prefect_future.result()

dask_flow()


17:44:38.495 | INFO    | prefect.engine - Created flow run 'dramatic-reindeer' for flow 'dask-flow'

17:44:38.525 | INFO    | prefect.task_runner.dask - Connecting to an existing Dask cluster at tcp://localhost:8786

17:44:40.830 | ERROR   | Flow run 'dramatic-reindeer' - Encountered exception during execution: RuntimeError('Found incompatible versions: client: 3.1.2, server: 2.14.2. Major versions must match.')
Traceback (most recent call last):
  File "/Users/sanjeevchauhan/miniforge3/envs/local/lib/python3.11/site-packages/prefect/flow_engine.py", line 655, in run_context
    yield self
  File "/Users/sanjeevchauhan/miniforge3/envs/local/lib/python3.11/site-packages/prefect/flow_engine.py", line 699, in run_flow_sync
    engine.call_flow_fn()
  File "/Users/sanjeevchauhan/miniforge3/envs/local/lib/python3.11/site-packages/prefect/flow_engine.py", line 678, in call_flow_fn
    result = call_with_parameters(self.flow.fn, self.parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sanjeevchauhan/miniforge3/envs/local/lib/python3.11/site-packages/prefect/utilities/callables.py", line 206, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/var/folders/mb/7p627zbj11v_2ym8j0sj95tw0000gn/T/ipykernel_82929/2766255537.py", line 15, in dask_flow
    return prefect_future.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sanjeevchauhan/miniforge3/envs/local/lib/python3.11/site-packages/prefect_dask/task_runners.py", line 132, in result
    future_result = self._wrapped_future.result(timeout=timeout)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sanjeevchauhan/miniforge3/envs/local/lib/python3.11/site-packages/distributed/client.py", line 322, in result
    return self.client.sync(self._result, callback_timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sanjeevchauhan/miniforge3/envs/local/lib/python3.11/site-packages/prefect_dask/client.py", line 62, in wrapper_func
    return run_task_sync(*args, **kwargs)
    ^^^^^^^^^^^^^^^^^
  File "/global/homes/s/sanjeevc/.conda/envs/sfapi_dask_env/lib/python3.11/site-packages/prefect/task_engine.py", line 1333, in run_task_sync
  File "/global/homes/s/sanjeevc/.conda/envs/sfapi_dask_env/lib/python3.11/contextlib.py", line 137, in __enter__
  File "/global/homes/s/sanjeevc/.conda/envs/sfapi_dask_env/lib/python3.11/site-packages/prefect/task_engine.py", line 732, in start
  File "/global/homes/s/sanjeevc/.conda/envs/sfapi_dask_env/lib/python3.11/contextlib.py", line 137, in __enter__
  File "/global/homes/s/sanjeevc/.conda/envs/sfapi_dask_env/lib/python3.11/site-packages/prefect/task_engine.py", line 655, in initialize_run
  File "/global/homes/s/sanjeevc/.conda/envs/sfapi_dask_env/lib/python3.11/contextlib.py", line 137, in __enter__
  File "/global/homes/s/sanjeevc/.conda/envs/sfapi_dask_env/lib/python3.11/site-packages/prefect/context.py", line 236, in get_or_create
  File "/global/homes/s/sanjeevc/.conda/envs/sfapi_dask_env/lib/python3.11/site-packages/prefect/context.py", line 218, in __enter__
  File "/global/homes/s/sanjeevc/.conda/envs/sfapi_dask_env/lib/python3.11/site-packages/prefect/client/orchestration.py", line 3744, in raise_for_api_version_mismatch
RuntimeError: Found incompatible versions: client: 3.1.2, server: 2.14.2. Major versions must match.

17:44:40.979 | ERROR   | Flow run 'dramatic-reindeer' - Finished in state Failed('Flow run encountered an exception: RuntimeError: Found incompatible versions: client: 3.1.2, server: 2.14.2. Major versions must match.')

RuntimeError: Found incompatible versions: client: 3.1.2, server: 2.14.2. Major versions must match.